In [1]:
import sys
sys.path.insert(0,'/home/jovyan/work/src/')
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import sessionmaker
from arise.barcode.metadata.orm.barcode import Barcode
from arise.barcode.metadata.orm.marker import Marker
from arise.barcode.metadata.orm.nsr_species import NsrSpecies
from arise.barcode.metadata.orm.nsr_synonym import NsrSynonym
from arise.barcode.metadata.orm.nsr_node import NsrNode
from arise.barcode.metadata.orm.specimen import Specimen
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
from ete3 import Tree, TreeStyle
import os
from ipywidgets import IntProgress
from IPython.display import display

os.environ['QT_QPA_PLATFORM']='offscreen'  # to avoid to kill the kernel using ete render
os.environ['XDG_RUNTIME_DIR']='/tmp/runtime-jovyan'

In [2]:
engine = create_engine('sqlite:////home/jovyan/work/data/sqlite/arise-barcode-metadata.db', echo=False)

# make session
Session = sessionmaker(engine)
session = Session()

In [3]:
# here we go and traverse the tree from the root
nsr_root = NsrNode.get_root(session)
print(nsr_root.name)

All of life


In [4]:

tree2 = Tree()
ete_root2 = tree2.add_child(name = nsr_root.name)
ete_root2.add_feature('rank','life')

tree2 = nsr_root.to_ete(session, 'family')
print('tree2.get_children()', len(tree2.get_children()))
descendants = tree2.get_descendants()
print('tree2.get_descendants()', len(descendants))
from collections import Counter
print(Counter([e.rank for e in descendants]))
print('tree2.get_descendants() family', len([e for e in tree2.get_descendants() if e.rank == 'family'])) # why not 4268?
print('(NsrNode.rank == family).count()', session.query(NsrNode).filter(NsrNode.rank == 'family').count()) # 4268



# debuging
query = session.query(Specimen.specimen_id, Barcode.barcode_id, Specimen.species_id).join(Barcode)
print(query)
d2 = {}
for specimen_id, barcode_id, species_id in query.all():
    if species_id not in d2:
        d2[species_id] = { 'specimens': {}, 'barcodes_count': 0 }
    if specimen_id not in d2[species_id]['specimens']:
        d2[species_id]['specimens'][specimen_id] = set()
    if barcode_id not in d2[species_id]['specimens'][specimen_id]:
        d2[species_id]['specimens'][specimen_id].add(barcode_id)
        d2[species_id]['barcodes_count'] += 1
        # if (len(d2[species_id]['specimens']) > 1 and len(d2[species_id]['specimens'][specimen_id]) > 1):
        #     print(species_id)
        #     break

print(d2[25384])

from sqlalchemy import func
# prefetch number of barcodes per species_id
query = session.query(Specimen.species_id, func.count()).join(Barcode).group_by(Specimen.species_id)
# print(query)
species_bc_dict = { e:c for e, c in query.all() }
# print(species_bc_dict[25384])


all_bc = 0
for node in tree2.iter_descendants(strategy='postorder'):
    if node.rank == 'family':
        tips = session.query(NsrNode).filter(NsrNode.id == node.id).first().get_descendants(session).all()
        total_bc = 0
        for tip in tips:
            if tip.species_id and tip.species_id in species_bc_dict:
                total_bc += species_bc_dict[tip.species_id]
        node.add_feature('total_sp', len(tips))
        node.add_feature('total_bc', total_bc)   
        all_bc += total_bc
    else:
        total_sp = 0
        total_bc = 0
        for child in node.get_children():
            total_sp += child.total_sp
            total_bc += child.total_bc
        node.add_feature('total_sp', total_sp)
        node.add_feature('total_bc', total_bc)   


ete_root2 = tree2.get_tree_root().get_children()[0]
print('ete_root2.total_sp', ete_root2.total_sp)
print('ete_root2.total_bc', ete_root2.total_bc) # why not 128195 = # of barcodes in db

tree2.get_children() 1
tree2.get_descendants() 9222
Counter({'order': 4221, 'class': 2025, 'phylum': 1530, 'kingdom': 1440, None: 6})
tree2.get_descendants() family 0
(NsrNode.rank == family).count() 4268
SELECT specimen.specimen_id AS specimen_specimen_id, barcode.barcode_id AS barcode_barcode_id, specimen.species_id AS specimen_species_id 
FROM specimen JOIN barcode ON specimen.specimen_id = barcode.specimen_id
{'specimens': {677: {682}, 956: {963}, 1222: {1235, 1236}, 1863: {1884, 1885}, 13536: {13666}, 20210: {20437}, 22094: {22340}, 26356: {26663}, 27731: {28063}, 28189: {28525}, 31089: {31450}, 36116: {36529}, 42646: {43128}, 42919: {43401}, 44135: {44637}, 44385: {44889}, 44465: {44969}, 45399: {45921}, 46683: {47216}, 52675: {53285}, 57150: {57834}, 58686: {59390}, 61170: {61918}, 63527: {64314}, 65782: {66597}, 70994: {71859}, 78599: {79555}, 83526: {84534}, 90757: {91872}, 92617: {93759}, 95726: {96900}, 99348: {100560}, 101010: {102240}, 105078: {106356}, 116417: {117869}, 1